Objetive of the project could be finding a regularisation of the latent space to extract high-level features of the signals obtained from the Sauron Semiconductors placed inside the AGATA.

Things to do:
-Solve errors of Calibration_modified.py.
-Find regularisation strategies from the book ML for Physicist in this unsupervised autoencoder.
-Add .root dataset of Daniele Mengoni to the code.
-Improve the noise reduction by including other techniques to the dataset (e.g. cross-validation).
-Add axis units.


In [15]:
#Loading the libraries and compiling the c++ module

from torch.utils.cpp_extension import load
import torch
import torch.nn as nn
import matplotlib.pyplot as plt
#from sklearn.metrics import confusion_matrix
import numpy as np
#import seaborn as sn
import os
import glob
import lightning
import matplotlib
import math
import torchmetrics
import pytorch_model_summary as pms


#from autoencoder import *

#Compile the c++ module
pid_dataset = load(name="PIDDataset",
                   sources=["PidDataset.cpp","ReadRawData.cpp","ReadBinaryData.cpp", "mwdlib.cpp"],
                   verbose=True,
                   extra_cflags=['-O3'],
                  )

# TO SHOW INTERACTIVE PLOT
%matplotlib widget

torch.set_float32_matmul_precision('medium')

Using /home/luna/.cache/torch_extensions/py311_cu118 as PyTorch extensions root...
No modifications detected for re-loaded extension module PIDDataset, skipping build step...
Loading extension module PIDDataset...


In [16]:
#Define NN parameters


derivative_order=1
normalized=False
convolutional=False #I can try to put this to False!!!
dropout_p=0.0 #In big NN we can put this parameter to 50% to avoid overfitting, by eliminating neurons. In this case, a smaller NN, it is not that useful.
#nLayers_list = [20, 30, 40, 50]
#nLayers_list = [4,6,8,10, 20, 60, 100]
nLayers_list=[4]
#act_fn = torch.nn.ReLU
kernel_size = 6
stride = 2
act_fn = torch.nn.GELU #Activation function
#pooling_kernel_size = 2

#Training parameters
batch_size = 150 #1000 was working. 
learning_rate = 3e-3 #If it's too big, it may go out of the gradient descent.
#batch_size = 5000 #this only makes things slow
retrain=True #??
version_nr = 0
EarlyStoppingNr = 10 #15 was working.
max_epochs = 100 #before we had 300.
exclude_outliers = False
hidden_size = 128  # Número de neuronas en cada capa oculta de la LSTM




#Dataset
dataset = "Fazia"

if dataset == "Sauron":
    trainDatasetFile = "./DataFusEvSauron/RU_caendig_i1468_0005_0000.caendat"
    datasetEvents = 3_000_000 #Events=Number of total signals.
    samples_number=128
    first_sample=0
    n_channels=64
elif dataset == "Oscar":
    trainDatasetFile = "./DataLiFOscar/data_0-7.caendat"
    datasetEvents = 709_755
    samples_number=40
    first_sample=200
    n_channels=16
elif dataset == "Fazia":
    trainDatasetFile = "./Fazia/Signal_and_IdCal_221.dat"
    datasetEvents = 709_756
    samples_number=30
    first_sample=10
    n_channels=25



In [17]:

class LSTMNet(lightning.LightningModule):
    def __init__(self, input_size, hidden_size, num_layers, act_fn, num_classes, dropout_p=0.5, class_weights=None):
        super().__init__()
        self.accuracy = torchmetrics.Accuracy(task='multiclass', num_classes=num_classes)
        if class_weights is not None:
            self.criterion = torch.nn.CrossEntropyLoss(weight=class_weights)
        else:
            self.criterion = torch.nn.CrossEntropyLoss()
        
        self.num_classes = num_classes
        self.hidden_size = hidden_size
        self.num_layers = num_layers

        # Definir la red LSTM
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True, dropout=dropout_p)

        # Fully connected layer para la salida
        self.fc = nn.Linear(hidden_size, num_classes)

    def forward(self, x):
    # Supongamos que x tiene las dimensiones (batch_size, sequence_length, channels, input_size)
        if len(x.shape) == 4:
            batch_size, sequence_length, channels, input_size = x.shape
            # Aplanar las dimensiones de 'channels' y 'input_size' para que sea compatible con LSTM
            x = x.view(batch_size, sequence_length, channels * input_size)

        # Inicializar los estados ocultos y de celda
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)
        c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)

        # Pasar por la capa LSTM
        out, _ = self.lstm(x, (h0, c0))

        # Usar la salida de la última secuencia temporal
        out = self.fc(out[:, -1, :])
        return out


    def _get_reconstruction_loss(self, batch):
        x, y = batch
        x_hat = self.forward(x)
        loss = self.criterion(x_hat, y)
        return loss

    def configure_optimizers(self):
        optimizer = torch.optim.Adam(self.parameters(), lr=1e-3)
        scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode="min", factor=0.2, patience=3, min_lr=5e-7)
        return {"optimizer": optimizer, "lr_scheduler": scheduler, "monitor": "val_loss"}

    def training_step(self, batch, batch_idx):
        x, y = batch
        x_hat = self(x)
        loss = self.criterion(x_hat, y)
        acc = self.accuracy(x_hat, y)
        self.log('train_loss', loss, on_step=True, on_epoch=True)
        self.log('train_acc', acc, on_step=True, on_epoch=True)
        return loss

    def validation_step(self, batch, batch_idx):
        x, y = batch
        x_hat = self(x)
        loss = self.criterion(x_hat, y)
        acc = self.accuracy(x_hat, y)
        
        self.log('val_loss', loss, on_step=False, on_epoch=True)
        self.log('val_acc', acc, on_step=False, on_epoch=True)
        return {'val_loss': loss, 'val_acc': acc, 'preds': x_hat, 'targets': y}

    def test_step(self, batch, batch_idx):
        x, y = batch
        x_hat = self(x)
        loss = self.criterion(x_hat, y)
        acc = self.accuracy(x_hat, y)
        self.log('test_loss', loss, on_step=False, on_epoch=True)
        self.log('test_acc', acc, on_step=False, on_epoch=True)
        return {'test_loss': loss, 'test_acc': acc, 'preds': x_hat, 'targets': y}



In [18]:
#Load the dataset


split_percent = [0.8, 0.1, 0.1] #90% for training, 5% for testing and 5% for validating.

[train_dataset, val_dataset, test_dataset] = torch.utils.data.random_split(
                pid_dataset.PIDDataset(root=trainDatasetFile, 
                                       mode=pid_dataset.kTrain, 
                                       count=datasetEvents, 
                                       nder=derivative_order, 
                                       nsamples=samples_number, 
                                       firstsample=first_sample,
                                       nchannels=n_channels,
                                       normalized=normalized,
                                       discardenergies=False,
                                       minenergy=0,
                                       maxenergy=100,),
                                       split_percent)

#Sampler to normalize the amount of types of each class
weights = torch.zeros(len(train_dataset))
class_counts = torch.zeros(n_channels)

#cont the occurrence of each class in the dataset
for i in range(len(train_dataset)):
    class_counts[train_dataset[i][1]] += 1

#calculate the weights
for i in range(len(train_dataset)):
    weights[i] = class_counts[train_dataset[i][1]]
    
weights = 1.0 / weights
weights = weights / weights.sum()

sampler = torch.utils.data.WeightedRandomSampler(weights=weights, num_samples=len(train_dataset), replacement=True)

train_loader = torch.utils.data.DataLoader(train_dataset, 
                                           batch_size=batch_size, 
                                           #shuffle=True, 
                                           drop_last=True, 
                                           pin_memory=True, 
                                           num_workers=4,
                                           sampler=sampler)
#Batch size: number of signals per batch.
#Shuffle: shuffle data at every epoch.
#Epoch: During an epoch, the model processes each training example once, 
#and the learning process updates the model's parameters based on the loss calculated from the training examples.

val_loader = torch.utils.data.DataLoader(val_dataset, 
                                         batch_size=2_000, 
                                         shuffle=False, 
                                         drop_last=False, 
                                         num_workers=4)
test_loader = torch.utils.data.DataLoader(test_dataset, 
                                          batch_size=2_000, 
                                          shuffle=False, 
                                          drop_last=False, 
                                          num_workers=4)

def get_train_images(num):
    #return [torch.stack([train_dataset[i][0] for i in range(num)], dim=0), torch.stack([train_dataset[i][1] for i in range(num)], dim=0)]
    return torch.stack([train_dataset[i][0] for i in range(num)], dim=0)
    #Here i is the batch and what is 0? dim=0 is the dimension of the output tensor.
    #What are IDs???

def get_train_labels(num):
    #return [torch.stack([train_dataset[i][0] for i in range(num)], dim=0), torch.stack([train_dataset[i][1] for i in range(num)], dim=0)]
    return torch.stack([train_dataset[i][1] for i in range(num)], dim=0)
    #Here i is the batch and what is 0? dim=0 is the dimension of the output tensor.
    #What are IDs???

def get_validation_images(num):
    #return [torch.stack([val_dataset[i][0] for i in range(num)], dim=0), torch.stack([val_dataset[i][1] for i in range(num)], dim=0)]
    return torch.stack([val_dataset[i][0] for i in range(num)], dim=0)

def get_validation_labels(num):
    #return [torch.stack([val_dataset[i][0] for i in range(num)], dim=0), torch.stack([val_dataset[i][1] for i in range(num)], dim=0)]
    return torch.stack([val_dataset[i][1] for i in range(num)], dim=0)
    
def get_test_images(num):
    #return [torch.stack([test_dataset[i][0] for i in range(num)], dim=0), torch.stack([test_dataset[i][1] for i in range(num)], dim=0)]
    return torch.stack([test_dataset[i][0] for i in range(num)], dim=0)


In [ ]:
#Debug cell???

# Supongamos que nLayers_list es una lista de tamaños para las capas ocultas
for nLayers in nLayers_list:
    input_size = derivative_order * samples_number + 1 # Ajusta esto de acuerdo a tus datos de entrada

    # Crea la red totalmente conectada con el tamaño de las capas ocultas basadas en nLayers
    test = LSTMNet(input_size=input_size, 
               num_classes=n_channels, 
               hidden_size=hidden_size,  # Tamaño de la capa oculta
               num_layers=nLayers,  # Número de capas LSTM
               act_fn=nn.GELU, 
               dropout_p=0.5, 
               class_weights=None)
   

    x = test(get_train_images(4))  # x es la salida de la red
    print(pms.summary(test, get_train_images(4)))  # Resumen de la arquitectura


---------------------------------------------------------------------------------------------
      Layer (type)                              Output Shape         Param #     Tr. Param #
            LSTM-1     [4, 1, 128], [4, 4, 128], [4, 4, 128]         478,720         478,720
          Linear-2                                   [4, 25]           3,225           3,225
Total params: 481,945
Trainable params: 481,945
Non-trainable params: 0
---------------------------------------------------------------------------------------------


In [ ]:
#Callback functions

CHECKPOINT_PATH = os.environ.get("PATH_CHECKPOINT", "saved_models/classifier")

class ConfusionMatrixCallback(lightning.pytorch.callbacks.Callback):
    def __init__(self, input, labels, every_n_epochs=1, data_type="train"):
        super().__init__()# It's often used to call the parent class's __init__ method to ensure that the parent class is properly initialized when creating an instance of the derived class.
        # Only save uthose images every N epochs (otherwise tensorboard gets quite large)
        self.every_n_epochs = every_n_epochs
        self.input = input
        self.labels = labels
        self.data_type = data_type

    
    def on_validation_epoch_end(self, trainer, pl_module):
        signals = self.input.to(pl_module.device)
        with torch.no_grad():
            pl_module.eval()
            print(signals.shape)
            output = pl_module(signals)
            pl_module.train()

        #calculate te confusion matrix
        cm = torchmetrics.ConfusionMatrix(num_classes=pl_module.num_classes, task='multiclass')
        cm.update(output, self.labels)

        fig, ax = plt.subplots(figsize=(15, 15))
        cm = cm.compute().numpy()
        cax = ax.matshow(cm, cmap='Blues')
        plt.colorbar(cax)
        for (i, j), val in np.ndenumerate(cm):
            ax.text(j, i, f'{val}', ha='center', va='center', color='black')

        #ax.imshow(cm.compute().numpy(), cmap='hot', interpolation='nearest')
        ax.set_title('Confusion matrix')
        ax.set_xlabel('Predicted')
        ax.set_ylabel('True')
        if self.data_type == "train":
            trainer.logger.experiment.add_figure("Confusion Matrix Train", fig, global_step=trainer.global_step)
        elif self.data_type == "validation":
            trainer.logger.experiment.add_figure("Confusion Matrix Validation", fig, global_step=trainer.global_step)


In [ ]:
#Training function definition

def train(base_channel_size, retrain=False):
    # Create a PyTorch Lightning trainer with the generation callback

    netName = "model_bc%i_do%i" % (base_channel_size, derivative_order)
    
    callbacks=[
        #lightning.pytorch.callbacks.ModelCheckpoint(save_weights_only=True),
        #ImageCallback(every_n_epochs=1),
        ConfusionMatrixCallback(input=get_train_images(40_000), labels=get_train_labels(40_000), every_n_epochs=1, data_type="train"),
        ConfusionMatrixCallback(input=get_validation_images(10_000), labels=get_validation_labels(10_000), every_n_epochs=1, data_type="validation"),
        lightning.pytorch.callbacks.LearningRateMonitor("epoch"),
        lightning.pytorch.callbacks.ModelCheckpoint(
            monitor='val_loss',
            mode='min',
            save_top_k=1,  # Save only the best checkpoint based on validation loss
            save_last=True,  # Save the latest checkpoint
            filename='{epoch}-{val_loss:.2f}',  # Customize the filename with epoch and validation loss
            verbose=True,
        ),
        lightning.pytorch.callbacks.EarlyStopping(
            monitor='val_loss',
            mode='min',
            patience=EarlyStoppingNr,
            verbose=False,
            min_delta=0.00,
        ),
        # Other callbacks...
    ]

    trainer = lightning.Trainer(
        default_root_dir=os.path.join(CHECKPOINT_PATH, netName),
        accelerator="auto",
        devices=1,
        callbacks=callbacks,
        max_epochs=max_epochs,
        logger=lightning.pytorch.loggers.tensorboard.TensorBoardLogger("saved_models/classifier", 
                                                                       name=netName),
    )
    trainer.logger._log_graph = True  # If True, we plot the computation graph in tensorboard
    trainer.logger._default_hp_metric = None  # Optional logging argument that we don't need

    # Check whether pretrained model exists. If yes, load it and skip training
    #ckpt_files = glob.glob('saved_models/autoencoder/%s/version_0/checkpoints/epoch*.ckpt' % (netName))
    ckpt_files = glob.glob('saved_models/classifier/%s/version_%i/checkpoints/epoch*.ckpt' % (netName, version_nr))

    if ckpt_files and not retrain:
        print("Found pretrained model, called %s" % ckpt_files[0])
        model = LSTMNet.load_from_checkpoint(ckpt_files[0])
        #model.to("cuda")
        model.to("cpu")
    else:   
        #ids_nr = get_test_images(1)[1].shape[1] - 1 
       # Dentro de la función train, cambia la siguiente línea
        model = LSTMNet(input_size=input_size,  
                num_classes=n_channels,
                hidden_size=hidden_size,  # Aquí puedes definir el tamaño de las capas ocultas
                num_layers=nlayers,  # Esto ajusta el número de capas
                act_fn=act_fn, 
                dropout_p=dropout_p,
                class_weights=None)

                        
        print("testing on a single batch...")
        print("inished testing on a single batch...")

        trainer.fit(model, train_loader, val_loader)

        print("Finished training, saving model...")

        # Test best model on validation and test set
        val_result = trainer.test(model, dataloaders=val_loader, verbose=False)
        test_result = trainer.test(model, dataloaders=test_loader, verbose=False)
        result = {"test": test_result, "val": val_result}
        return model, result
        print("Finished training, saving model...")

    # Test best model on validation and test set
    val_result = trainer.test(model, dataloaders=val_loader, verbose=False)
    test_result = trainer.test(model, dataloaders=test_loader, verbose=False)
    result = {"test": test_result, "val": val_result}
    return model, result

In [ ]:
#Training loop
colors = ["b", "g", "r", "c", "m", "y", "k", "w", "orange", "purple"]
model_list= []
for nlayers in nLayers_list:
    model_ld, result_ld = train(nlayers, retrain=retrain)
    model_list.append({"nlayers": nlayers,
                       "model": model_ld, 
                       "result": result_ld, 
                       "color": colors.pop(0)})
    

/home/luna/miniconda3/envs/pytorch/lib/python3.11/site-packages/torch/cuda/__init__.py:611: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")
GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs

  | Name      | Type               | Params
-------------------------------------------------
0 | accuracy  | MulticlassAccuracy | 0     
1 | criterion | CrossEntropyLoss   | 0     
2 | lstm      | LSTM               | 478 K 
3 | fc        | Linear             | 3.2 K 
-------------------------------------------------
481 K     Trainable params
0         Non-trainable params
481 K     Total params
1.928     Total estimated model params size (MB)


testing on a single batch...
inished testing on a single batch...


/home/luna/miniconda3/envs/pytorch/lib/python3.11/site-packages/lightning/pytorch/loggers/tensorboard.py:187: Could not log computational graph to TensorBoard: The `model.example_input_array` attribute is not set or `input_array` was not given.


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

torch.Size([40000, 1, 1, 31])
torch.Size([10000, 1, 1, 31])


...

torch.Size([40000, 1, 1, 31])
torch.Size([10000, 1, 1, 31])


Epoch 65, global step 249810: 'val_loss' was not in top 1


Finished training, saving model...


Testing: |          | 0/? [00:00<?, ?it/s]

Testing: |          | 0/? [00:00<?, ?it/s]

In [ ]:
 #Compare loss as a function of latent dimensionality

latent_dim=n_channels

latent_dims = [m["latent_dim"] for m in model_list]
nlayers = [m["nlayers"] for m in model_list]
val_scores = [m["result"]["val"][0]["test_loss"] for m in model_list]
colors = [m["color"] for m in model_list]

fig, axes = plt.subplots(1, 3, figsize=(16, 4))
axes[0].scatter(latent_dims, val_scores, color=colors, marker="*", s=100)
axes[0].set_xlabel("Latent Dimensionality")
axes[0].set_ylabel("Validation Loss")

axes[1].scatter(nlayers, val_scores, color=colors, marker="*", s=100)
axes[1].set_xlabel("Number of Layers")
axes[1].set_ylabel("Validation Loss")

# Scatter plot with latent_dim on x-axis, nlayers on y-axis, and colormap on z-axis (val_scores)
scatter = axes[2].scatter(latent_dims, nlayers, c=val_scores, cmap='viridis', marker="s", s=600)
axes[2].set_xlabel("Latent Dimensionality")
axes[2].set_ylabel("Number of Layers")
axes[2].set_title("Validation Loss")

# Add colorbar
cbar = fig.colorbar(scatter, ax=axes[2])
cbar.set_label("Validation Loss")
plt.show()

KeyError: 'latent_dim'

No such comm: 636b9c8c87684d6b861ed8a6b9838099
No such comm: 636b9c8c87684d6b861ed8a6b9838099
No such comm: 636b9c8c87684d6b861ed8a6b9838099
No such comm: 636b9c8c87684d6b861ed8a6b9838099
No such comm: 636b9c8c87684d6b861ed8a6b9838099
No such comm: 636b9c8c87684d6b861ed8a6b9838099
No such comm: 636b9c8c87684d6b861ed8a6b9838099
No such comm: 636b9c8c87684d6b861ed8a6b9838099
No such comm: 636b9c8c87684d6b861ed8a6b9838099
No such comm: 636b9c8c87684d6b861ed8a6b9838099
No such comm: 636b9c8c87684d6b861ed8a6b9838099
No such comm: bee4231bdca542f7bf01d57d0c1f5445
No such comm: bee4231bdca542f7bf01d57d0c1f5445
No such comm: bee4231bdca542f7bf01d57d0c1f5445
No such comm: bee4231bdca542f7bf01d57d0c1f5445
No such comm: bee4231bdca542f7bf01d57d0c1f5445
No such comm: bee4231bdca542f7bf01d57d0c1f5445
No such comm: bee4231bdca542f7bf01d57d0c1f5445
No such comm: bee4231bdca542f7bf01d57d0c1f5445
No such comm: bee4231bdca542f7bf01d57d0c1f5445
No such comm: bee4231bdca542f7bf01d57d0c1f5445
No such comm:

In [ ]:
#Define plot functions

def plot(inTensor, axes, color, dim, label, linestyle="--", skip=0):
    for i, ax in enumerate(axes.flat):
        if i >= len(inTensor)+skip:
            break
        if i < skip:
            continue    
        ax.plot(inTensor[i-skip][0][dim].numpy(), linestyle, color=color, label=label)
        ax.legend()

def reconstruct_signals(input_signal, model_l, axes, dim):
    # Reconstruct images
    model = model_l["model"].eval()
    color = model_l["color"]
    label = "ld %s nl %s" % (model_l["latent_dim"], model_l["nlayers"])
    with torch.no_grad():
        reconst_imgs = model(input_signal[0].to(model.device), input_signal[1].to(model.device))    
    reconst_imgs = reconst_imgs.cpu()
    plot(reconst_imgs, axes, color, dim, label)

def generate_signals(input_latents, model_l, axes, dim, skip=0, color="b"):
    # Reconstruct images
    model = model_l["model"].eval()
    label = "e=%.1f, x=%.1f, y=%.1f" % (input_latents[1][0][0].numpy(), input_latents[0][0][0].numpy(), input_latents[0][0][1].numpy())
    with torch.no_grad():
        reconst_imgs = model.decoder(input_latents[0].to(model.device), input_latents[1].to(model.device))    
    reconst_imgs = reconst_imgs.cpu()
    plot(reconst_imgs, axes, color, dim, label, skip=skip)

In [ ]:
#Compare the reconstructions with different model hyper-parameters
ncols = 4
nrows = 16
dim = 0

fig, axes = plt.subplots(nrows, ncols, figsize=(15, 40))
input_imgs = get_train_images(ncols*nrows)
#input_imgs = get_validation_images(ncols*nrows)

for m in model_list:
    reconstruct_signals(input_imgs, m, axes, dim)
plot(input_imgs[0], axes, "black", dim=dim, label="input", linestyle="-")

plt.show()

In [ ]:
#Visualize the latent space

import ipywidgets as widgets
import ipywidgets as widgets
    
# Create the slider
ene_slider = widgets.FloatSlider(value=0.5, min=0, max=1, step=0.01, description='Energy:')
nch = get_train_images(1)[1].shape[1] - 1
ch_dropdown = widgets.Dropdown(
    options=[(f'Channel {i}', i) for i in range(-1,nch+1)],
    value=0,
    description='Channel:',
)
# Display the slider
display(ch_dropdown)
display(ene_slider)

idx_test = 0

def get_histo(idx_ch):
    with torch.no_grad():
        model_list[idx_test]["model"].eval()
        dataset = get_train_images(500_000)
        # Filter the dataset
        if idx_ch >= 0:
            dataset = (dataset[0][dataset[1][:, idx_ch+1] == 1], 
                       dataset[1][dataset[1][:, idx_ch+1] == 1])
        avg_id = dataset[1].mean(dim=0)

        

        histoOut = torch.histogramdd(   model_list[idx_test]["model"].forward(dataset[0].to(model_list[idx_test]["model"].device)),
                                        bins=5_000,
                                        range = None,
                                        )
        histoOut[0].detach().numpy()

        return [histoOut, avg_id]


#Create the 2 canvases
fig, axes = plt.subplots(1, 2, figsize=(15, 10))

                
def on_ch_dropdown_change(change):
    value = change['new']
    histo, avg_id = get_histo(value)

    colors=["b", "g", "r", "c", "m", "y", "k", "w", "orange", "purple", "brown", "pink", "olive", "cyan"]

    axes[0].imshow( histo[0], 
                    extent=[histo[1][0][0], 
                            histo[1][0][-1],
                            histo[1][1][0], 
                            histo[1][1][-1]], 
                    aspect='auto', 
                    origin='lower', 
                    norm=matplotlib.colors.LogNorm())
                    
    def on_histo_double_click(event):
        if event.dblclick:
            x = event.xdata
            y = event.ydata
            if x is not None and y is not None:

                #Get coordinates
                last_click = [x, y] 
                print("[%.2f, %.2f]" % (x, y))

                color=colors.pop(0)
                #Annote in the plot
                axes[0].annotate("[%.2f, %.2f]" % (x, y), (x, y), color=color, bbox=dict(facecolor='white', edgecolor='white', alpha=0.5))
                axes[0].plot(x, y, 'o', color=color)

                #Plot the signal
                def on_ene_slider_change(change):
                    value = change['new']
                    double_click_tensor = torch.FloatTensor([last_click])
                    id_tensor = avg_id.unsqueeze(0).repeat(double_click_tensor.size(0), 1)
                    id_tensor[:, 0] = value
                    generate_signals([double_click_tensor, id_tensor], model_list[idx_test], axes, dim=0, skip=1, color=color)

                ene_slider.observe(on_ene_slider_change, names='value')

    fig.canvas.mpl_connect('button_press_event', on_histo_double_click)

ch_dropdown.observe(on_ch_dropdown_change, names='value')            



    

plt.show()

In [ ]:
#Finding similar signals in the latent space and then checking the signals

def embed_imgs(model, data_loader):
    # Encode all images in the data_laoder using model, and return both images and encodings
    img_list, embed_list = [], []
    model.eval()
    for imgs, ids in data_loader:
        with torch.no_grad():
            z = model.encoder(imgs.to(model.device), ids.to(model.device))
        img_list.append(imgs)
        embed_list.append(z)
    return (torch.cat(img_list, dim=0), torch.cat(embed_list, dim=0))


train_img_embeds = embed_imgs(model_list[idx_test]["model"], train_loader)
test_img_embeds = embed_imgs(model_list[idx_test]["model"], test_loader)

def find_similar_images(query_img, query_z, key_embeds, ax, K=8, dim=0):
    # Find closest K images. We use the euclidean distance here but other like cosine distance can also be used.
    dist = torch.cdist(query_z[None, :], key_embeds[1], p=2)
    dist = dist.squeeze(dim=0)
    dist, indices = torch.sort(dist)
    # Plot K closest images
    imgs_to_display = torch.cat([query_img[None], key_embeds[0][indices[:K]]], dim=0)
    #grid = torchvision.utils.make_grid(imgs_to_display, nrow=K + 1, normalize=True, value_range=(-1, 1))
    #grid = grid.permute(1, 2, 0)
    #plt.figure(figsize=(12, 3))
    #plt.imshow(grid)
    #plt.axis("off")
    #plt.show()
    colors = ["b", "g", "r", "c", "m", "y", "k", "w", "orange", "purple", "brown", "pink"]
    ax.plot(query_img[0][dim].numpy(), color="k", label="query")
    for i in range(imgs_to_display.size()[0]):
        ax.plot(imgs_to_display[i][0][dim].numpy(), "--", color=colors.pop(0), label="latent_dim %i" % latent_dim)

In [ ]:
# Plot the closest images for the first N test images as example
ncols = 6
nrows = 6
fig, axes = plt.subplots(nrows, ncols, figsize=(10, 10))
for i, ax in enumerate(axes.flat):
    find_similar_images(test_img_embeds[0][i], test_img_embeds[1][i], key_embeds=train_img_embeds, K=6, ax=ax, dim=0)

#This does not make sense, i probabily did something wrong